In [1]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output
import json

In [2]:
# Replace with the name you want used in publications of this data set
name = "Jenna Schabdach"

# Projects: SLIP, SLIP PreK, and SLIP Adolescents
project = "SLIP Adolescents"

# Replace with the path to your .json file containing a dictionary of all the phrases you'd like to be highlighted
phrasesToHighlightFn = "phrases_to_highlight.json"

# Load the dictionary of phrases to highlight in certain colors 
with open(phrasesToHighlightFn, 'r', encoding='utf-8') as f:
    toHighlight = json.load(f)
    
getGraderStatusReport(name)


151
151
Jenna Schabdach has graded 151 of 151 reliability reports
Jenna Schabdach has graded 35864 unique reports of 35864 assigned where
33843 have been given a grade of 0
962 have been given a grade of 1
1059 have been given a grade of 2


In [7]:
isOnMainTable = welcomeUser(name)

Welcome, Jenna Schabdach
151
151
Jenna Schabdach has graded 151 of 151 reliability reports
You currently have 80 ungraded reports to work on.


In [ ]:
batchSize = 10

for i in range(batchSize):
    clear_output()
    print(i)
    if isOnMainTable:
        markOneReportSQL(name, project, toHighlight)
    else:
        markSelfEvalReportSQL(name, toHighlight)


4
BRAIN MRI, WITHOUT CONTRAST:
CLINICAL INDICATION: SCD -SS with TCD with nonvisualization on
the left side - r/o silent iNFARCTion; Sickle-cell disorder
TECHNIQUE: 
Brain: Sagittal 3D T1 gradient echo with axial reformations,
axial and coronal TSE T2, axial FLAIR, arterial spin labeled
perfusion imaging, and axial diffusion imaging were performed on
a 3.0 Tesla system.
MRA: MRA examination of the brain was performed utilizing a 3
Tesla system via 3D time-of-flight intracranial magnetic
resonance angiography of the circle of Willis, with volume of
interest 3D reconstructions using the maximum intensity
projection method.
MRA was deemed medically necessary based on suspicion of silent
of the iNFARCT.
COMPARISON:  Brain MRI dated 12/11/2020
FINDINGS:
Brain:
No reduced diffusion to suggest acute iNFARCT.
No areas of abnormal parenchymal signal intensity.
Hyperperfusion is seen symmetrically in both thalami and medial
occipital lobes, DOUBTFUL CLINICAL SIGNIFICANCE.
Ventricular system norm

In [4]:
# If/when you run out of reports, you can uncomment the following line of code
#   and run this cell to get 50 more reports from the database. Make sure your
#   values for `project` and `queryFn` match the project you're working on.
# Projects: SLIP, SLIP PreK, and SLIP Adolescents
# Queries: ./queries/slip_base.txt, ./queries/start2018_ages2-5years.txt, and ./queries/start2023_ages12-20years.txt
project = "SLIP Adolescents"
queryFn = "./queries/start2023_ages12-20years.txt"
getMoreReportsToGrade(name, project, queryFn, 100)

It is expected for this function to take several minutes to run. Your patience is appreciated.
100 reports were added for grader Jenna Schabdach
